# 네이버 뉴스 크롤링

Last update: May 28. 2017  
Kyusik Kim

* 깔끔하지도 않고 오직 나만을 위해 만든 크롤러이기 때문에 몇 가지 TIP을 공유하는 차원에서 글을 쓴다. (공유라기 보다는 그냥 내가 까먹지 않으려고... 일을 그만 두면서 코드를 가지고 왔다고 생각했는데, 감쪽같이 사라져 새로 만드는 데 난감했기 때문이다)
* 먼저 크롤링(crawling)은 우리가 인터넷 서핑할 때 보는 웹 페이지 내에 있는 데이터를 추출해 내는 행위를 말하며, 이를 수행하는 일련의 시스템(?)을 크롤러(crawler)라 할 수 있겠다. 다양한 방법이 있지만, 나는 Python을 이용한다. 
* Python을 이용해 크롤링을 할 때 핵심이 되는 라이브러리를 열거하면, BeautifulSoup, requests, urllib.request 등이 있다. 보조적으로, pandas, re, time, random 을 이용한다.  
![그림1](https://github.com/iamglenn/mypython/blob/master/pic/pic_crawl_1.png?raw=true)


* 네이버 뉴스의 경우 기사 제목인 "포천시~ 참가" 부분과 빨간색 네모박스의 "국제뉴스" 부분의 url이 다르다. 둘의 차이를 설명하면, 기사 제목을 누르면 국제뉴스와 연결되고, 국제뉴스를 누르면 네이버뉴스 플랫폼에서 제공하는 국제뉴스의 기사에 연결된다는 점이 다르다. 이런 것은 좀 더 편하게 크롤링을 하기 위한 꼼수라고 해야 할까?
* 순서는 아래의 내용을 참고

* **import module**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import urllib.request
import re
import time
import random

* **네이버 뉴스 url 입력하기**

In [2]:
url_basic = "http://news.naver.com/main/search/search.nhn?query=%BE%C6%B4%C2%C7%FC%B4%D4&st=news.all&q_enc=EUC-KR&r_enc=UTF-8&r_format=xml&rp=none&sm=all.basic&ic=all&so=rel.dsc&rcnews=exist:032:005:086:020:021:081:022:023:025:028:038:469:&rcsection=exist:&stDate=range:20170521:20170528&detail=0&pd=3&r_cluster2_start=11&r_cluster2_display=10&start=11&display=10&startDate=2017-05-21&endDate=2017-05-28&page=1"
url_basic = url_basic[:-1]

* **총 페이지 수**

In [3]:
total_pages = 1

* **내보낼 때 파일 이름**

In [4]:
file_name = r'\아는형님'

In [5]:
article_url = []
press_name = []
press_time = []
press_title = []
press_text = []

* **url 가져오기**

In [6]:
url = url_basic + str(1)
print(url)

http://news.naver.com/main/search/search.nhn?query=%BE%C6%B4%C2%C7%FC%B4%D4&st=news.all&q_enc=EUC-KR&r_enc=UTF-8&r_format=xml&rp=none&sm=all.basic&ic=all&so=rel.dsc&rcnews=exist:032:005:086:020:021:081:022:023:025:028:038:469:&rcsection=exist:&stDate=range:20170521:20170528&detail=0&pd=3&r_cluster2_start=11&r_cluster2_display=10&start=11&display=10&startDate=2017-05-21&endDate=2017-05-28&page=1


In [7]:
source_code_from_URL = urllib.request.urlopen(url)
print(source_code_from_URL)

* **BeautifulSoup으로 가져오기**

In [8]:
soup = BeautifulSoup(source_code_from_URL, "lxml", from_encoding = "utf-8")
print(soup)

<!DOCTYPE HTML>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1023" name="viewport"/>
<meta content="뉴스검색 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="http://news.naver.com/main/search/search.nhn" property="og:url"/>
<meta content="http://static.news.naver.net/image/news/ogtag/navernews_200x200_20160804.png" property="og:image"/>
<meta content="네이버 뉴스 검색 서비스" property="og:description"/>
<meta content="summary" name="twitter:card"/>
<meta content="뉴스검색 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="http://static.news.naver.net/image/news/ogtag/navernews_200x200_20160804.png" name="twitter:image"/>
<meta content="네이버 뉴스 검색 서비스" name="twitter:description"/>
<title>아는형님 검색결과 : 네이버 뉴스</title>
<link href="http://static.news.naver.net/image/news/2014/favicon/favicon.ico" rel="s

* **div의 info class에 해당하는 내용 가져오기**

In [9]:
item = soup.find_all('div', 'info')
print(item)

[<div class="info">
<span class="pht"></span>
<span class="press">서울신문</span> <span class="bar"></span> <span class="time">2시간전</span> <span class="bar"></span> <a class="go_naver" href="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=106&amp;oid=081&amp;aid=0002824625" onclick="" target="_blank">네이버뉴스</a>
<!-- 개인화플러그인 -->
<div class="head_social share_area">
<span class="bar"></span>
<a class="btn_share btn_social naver-splugin nclicks(STA.share)" data-mail-display="off" data-me-display="off" data-oninitialize="splugin_oninitialize('this');" data-option="{baseElement:'spiButton0', layerPosition:'outside-bottom', align:'right', top:4, left:0, marginLeft:8, marginTop:10}" data-service-name="뉴스" data-source-name="서울신문" data-style="unity-v2" data-title="‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”" data-url="http://en.seoul.co.kr/news/newsView.php?id=20170528500034&amp;wlog_tag3=naver" href="#" id="spiButton0" title="소셜공유하기">
<span class="blind naver-splugin-c">소셜공유하기</span>
</a

* **div class="info"에서 a태그에 달려 있는 각 네이버뉴스의 url 주소 가져오기**

In [10]:
article = item[0].find('a')['href']
print(article)

http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=106&oid=081&aid=0002824625


In [11]:
news_url = urllib.request.urlopen(article)
print(news_url)

* **네이버뉴스에서 추출한 url에서 내용을 BeautifulSoup으로 가져오기**

In [12]:
soup2 = BeautifulSoup(news_url, "lxml", from_encoding = "utf-8")
print(soup2)

<!DOCTYPE html>
<html lang="ko">
<head>
<title id="browse_title">‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것” :: 네이버 TV연예</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script charset="utf-8" type="text/javascript">
var doc = document.documentElement;
doc.setAttribute('data-useragent', navigator.userAgent);
</script>
<link href="http://static.news.naver.net/image/news/2014/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<script src="http://static.entertain.naver.net/resources/20170522_165620/js/infra/jindo/jindo.desktop.ns.min.js" type="text/javascript"></script>
<script src="http://static.entertain.naver.net/resources/20170522_165620/js/infra/common/enter.define.js" type="text/javascript"></script>
<script src="http://static.entertain.naver.net/resources/20170522_165620/js/infra/image/imageError.js" type="text/javascript"></script>
<script type="text/javascript">
document.domain = 'naver.com';
</script>
<script type="text/javascript">
(f

* **네이버뉴스에서 div 태그에 있는 articeBody를 가져오기**
* **articeBody는 본문 내용이며, 이 html태그는 chrome에서 F12를 눌러 확인가능**
* **class 이름은 바뀔 수 있음을 유의. 예를 들면 세계일보는 articeBody, 조선일보는 articleBodyContents를 사용하기도 함**

In [13]:
article_body = soup2.find('div', id='articeBody')
print(article_body)

<div class="article_body font1 size3" id="articeBody">
<!-- [D] 본문 폰트 설정 class 안내
							나눔고딕 : font1
							맑은고딕 : font2
							돋움 : font3
							바탕 : font4
							폰트 사이즈 1 : size1
							폰트 사이즈 2 : size2
							폰트 사이즈 3 : size3
							폰트 사이즈 4 : size4
							폰트 사이즈 5 : size5
						-->
						
							
						
						[서울신문 En]<br/><span class="end_photo_org"> <img id="img1" onerror='javascript:this.src="http://static.news.naver.net/image/news/2009/press/empty.png";javascript:this.height="0px";javascript:imageErrorDetector(1, "http://mimgnews1.naver.net/image/081/2017/05/28/0002824625_001_20170528143116159.jpg");' src="http://mimgnews1.naver.net/image/081/2017/05/28/0002824625_001_20170528143116159.jpg?type=w540"/>
<em class="img_desc">‘아는형님’ 오현경</em></span>배우 오현경이 “강호동이 이상형이다”고 고백했다.<!-- MobileAdNew center --><br/><br/>오현경은 27일 JTBC ‘아는 형님’에 출연해 이같이 말했다. 강호동과 25년 지기 친구인 오현경은 “나보고 반한 적 없어?”라고 ‘기습’ 질문했다.<br/><br/>이에 강호동은 고개를 숙이고 손수건으로 이마 땀을 닦는 등 부끄러움을 감추지 못했다. 이어 오현경은 “만약 호동이가 대시했다면 사귀었을 것”

* **텍스트 부분만 가져오기**

In [14]:
dirt_text = article_body.find_all(text=True)
print(dirt_text)

['\n', ' [D] 본문 폰트 설정 class 안내\n\t\t\t\t\t\t\t나눔고딕 : font1\n\t\t\t\t\t\t\t맑은고딕 : font2\n\t\t\t\t\t\t\t돋움 : font3\n\t\t\t\t\t\t\t바탕 : font4\n\t\t\t\t\t\t\t폰트 사이즈 1 : size1\n\t\t\t\t\t\t\t폰트 사이즈 2 : size2\n\t\t\t\t\t\t\t폰트 사이즈 3 : size3\n\t\t\t\t\t\t\t폰트 사이즈 4 : size4\n\t\t\t\t\t\t\t폰트 사이즈 5 : size5\n\t\t\t\t\t\t', '\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t[서울신문 En]', ' ', '\n', '‘아는형님’ 오현경', '배우 오현경이 “강호동이 이상형이다”고 고백했다.', ' MobileAdNew center ', '오현경은 27일 JTBC ‘아는 형님’에 출연해 이같이 말했다. 강호동과 25년 지기 친구인 오현경은 “나보고 반한 적 없어?”라고 ‘기습’ 질문했다.', '이에 강호동은 고개를 숙이고 손수건으로 이마 땀을 닦는 등 부끄러움을 감추지 못했다. 이어 오현경은 “만약 호동이가 대시했다면 사귀었을 것”이라고 말했다.', '또한 오현경은 “사실 내 이상형은 강호동이다”고 말해 출연진을 당황하게 만들었다. 이어 그는 “지금 장난치는 거야. 너 결혼했는데 그러면 안된다”고 덧붙여 웃음을 자아냈다.', '한편 이날 강호동은 “오현경과 친구로 지낸지는 25년, 실제로 본 지는 28년 된다”며 오현경과의 우정을 과시했다.', '그는 “오현경이 (1989년) 미스코리아가 됐을 때 나도 백두장사가 됐다. 모 언론 인터뷰를 갔는데 그 때 엘리베이터에서 만났다”고 회상했다.', '사진 = 서울신문DB', ' MobileAdNew center ', '연예팀 seoulen@seoul.co.kr', '▶ [', '서울EN 바로가기', '] [', '페이스북', '] '

* **html 태그 없애는 정규표현식이었던듯..?**

In [15]:
cleaned_text = re.sub('[a-zA-Z]', '', str(dirt_text))
print(cleaned_text)

['\', ' [] 본문 폰트 설정  안내\\\\\\\\나눔고딕 : 1\\\\\\\\맑은고딕 : 2\\\\\\\\돋움 : 3\\\\\\\\바탕 : 4\\\\\\\\폰트 사이즈 1 : 1\\\\\\\\폰트 사이즈 2 : 2\\\\\\\\폰트 사이즈 3 : 3\\\\\\\\폰트 사이즈 4 : 4\\\\\\\\폰트 사이즈 5 : 5\\\\\\\', '\\\\\\\\\\\\\\\\\\\\\\\\\\\\\[서울신문 ]', ' ', '\', '‘아는형님’ 오현경', '배우 오현경이 “강호동이 이상형이다”고 고백했다.', '   ', '오현경은 27일  ‘아는 형님’에 출연해 이같이 말했다. 강호동과 25년 지기 친구인 오현경은 “나보고 반한 적 없어?”라고 ‘기습’ 질문했다.', '이에 강호동은 고개를 숙이고 손수건으로 이마 땀을 닦는 등 부끄러움을 감추지 못했다. 이어 오현경은 “만약 호동이가 대시했다면 사귀었을 것”이라고 말했다.', '또한 오현경은 “사실 내 이상형은 강호동이다”고 말해 출연진을 당황하게 만들었다. 이어 그는 “지금 장난치는 거야. 너 결혼했는데 그러면 안된다”고 덧붙여 웃음을 자아냈다.', '한편 이날 강호동은 “오현경과 친구로 지낸지는 25년, 실제로 본 지는 28년 된다”며 오현경과의 우정을 과시했다.', '그는 “오현경이 (1989년) 미스코리아가 됐을 때 나도 백두장사가 됐다. 모 언론 인터뷰를 갔는데 그 때 엘리베이터에서 만났다”고 회상했다.', '사진 = 서울신문', '   ', '연예팀 @..', '▶ [', '서울 바로가기', '] [', '페이스북', '] ', '▶ [', '스타 화보 보기', ']', 'ⓒ 서울신문(', '...', '), 무단전재 및 재배포금지\\\\\\']


* **\ 이런 것들 없애주는 정규표현식**

In [16]:
cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                              '', cleaned_text)
print(cleaned_text)

   본문 폰트 설정  안내나눔고딕  1맑은고딕  2돋움  3바탕  4폰트 사이즈 1  1폰트 사이즈 2  2폰트 사이즈 3  3폰트 사이즈 4  4폰트 사이즈 5  5 서울신문     ‘아는형님’ 오현경 배우 오현경이 “강호동이 이상형이다”고 고백했다     오현경은 27일  ‘아는 형님’에 출연해 이같이 말했다 강호동과 25년 지기 친구인 오현경은 “나보고 반한 적 없어”라고 ‘기습’ 질문했다 이에 강호동은 고개를 숙이고 손수건으로 이마 땀을 닦는 등 부끄러움을 감추지 못했다 이어 오현경은 “만약 호동이가 대시했다면 사귀었을 것”이라고 말했다 또한 오현경은 “사실 내 이상형은 강호동이다”고 말해 출연진을 당황하게 만들었다 이어 그는 “지금 장난치는 거야 너 결혼했는데 그러면 안된다”고 덧붙여 웃음을 자아냈다 한편 이날 강호동은 “오현경과 친구로 지낸지는 25년 실제로 본 지는 28년 된다”며 오현경과의 우정을 과시했다 그는 “오현경이 1989년 미스코리아가 됐을 때 나도 백두장사가 됐다 모 언론 인터뷰를 갔는데 그 때 엘리베이터에서 만났다”고 회상했다 사진  서울신문     연예팀  ▶  서울 바로가기   페이스북   ▶  스타 화보 보기  ⓒ 서울신문   무단전재 및 재배포금지


* **앞에 달려 있는 "본문 폰트 설정 안내나눔고딕"등의 글들을 없애주는 정규표현식**

In [17]:
cleaned_text = re.sub('본문 폰트 설정  안내나눔고딕  1맑은고딕  2돋움  3바탕  4폰트 사이즈 1  1폰트 사이즈 2  2폰트 사이즈 3  3폰트 사이즈 4  4폰트 사이즈 5  5',
                      '', cleaned_text)
print(cleaned_text)

    서울신문     ‘아는형님’ 오현경 배우 오현경이 “강호동이 이상형이다”고 고백했다     오현경은 27일  ‘아는 형님’에 출연해 이같이 말했다 강호동과 25년 지기 친구인 오현경은 “나보고 반한 적 없어”라고 ‘기습’ 질문했다 이에 강호동은 고개를 숙이고 손수건으로 이마 땀을 닦는 등 부끄러움을 감추지 못했다 이어 오현경은 “만약 호동이가 대시했다면 사귀었을 것”이라고 말했다 또한 오현경은 “사실 내 이상형은 강호동이다”고 말해 출연진을 당황하게 만들었다 이어 그는 “지금 장난치는 거야 너 결혼했는데 그러면 안된다”고 덧붙여 웃음을 자아냈다 한편 이날 강호동은 “오현경과 친구로 지낸지는 25년 실제로 본 지는 28년 된다”며 오현경과의 우정을 과시했다 그는 “오현경이 1989년 미스코리아가 됐을 때 나도 백두장사가 됐다 모 언론 인터뷰를 갔는데 그 때 엘리베이터에서 만났다”고 회상했다 사진  서울신문     연예팀  ▶  서울 바로가기   페이스북   ▶  스타 화보 보기  ⓒ 서울신문   무단전재 및 재배포금지


* **기사 제목을 가져오기 위해 p 태그에 있는 end_tit class를 가져오기**
* **텍스트 다듬는 것은 위와 동일**

In [18]:
title = soup2.find('p', attrs={'class':'end_tit'})
print(title)

<p class="end_tit">‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”</p>


In [19]:
title2 = title.find_all(text=True)
print(title2)                

['‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”']


In [20]:
title2 = re.sub('[a-zA-Z]', '', str(title2))
print(title2)               

['‘아는형님’ 오현경, “강호동이 이상형..고백했으면 사귀었을 것”']


In [21]:
title2 = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', title2)
print(title2)

‘아는형님’ 오현경 “강호동이 이상형고백했으면 사귀었을 것”


* **기사 정보들 가져오기**
* **신문사, 게시 시간 등**

In [22]:
a = item[0]

In [23]:
press = a.find('span', {'class': 'press'}).text
print(press)

서울신문


In [24]:
date = a.find('span',{'class':'time'}).text
print(date)

2시간전


In [33]:
article_url.append(article)
press_name.append(press)
press_time.append(date)  
press_title.append(title2)
press_text.append(cleaned_text)

* **Series로 만든 후 DataFrame으로 합친 결과를 내보내면 끝**

In [34]:
date = Series(press_time)
name = Series(press_name)
press_url = Series(article_url)
title_sr = Series(press_title)
text_sr = Series(press_text)

url_df = pd.concat([title_sr, name, date, text_sr, press_url], axis=1)
url_df.columns = ['TITLE', 'NAME', 'TIME', 'TEXT', 'URL']

In [35]:
pd.DataFrame(url_df)

,TITLE,NAME,TIME,TEXT,URL
0,‘아는형님’ 오현경 “강호동이 이상형고백했으면 사귀었을 것”,서울신문,"Series([], dtype: float64)",서울신문 ‘아는형님’ 오현경 배우 오현경이 “강호동이 이상형이다”고 ...,http://news.naver.com/main/read.nhn?mode=LSD&m...


## 완성!

In [40]:
for i in range(1, total_pages+1):
    url = url_basic + str(i)
    
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, "lxml", from_encoding = "utf-8")
    item = soup.find_all('div', 'info')
    
    for a in item:
        try:
            article = a.find('a')['href']
        
            news_url = urllib.request.urlopen(article)
            soup2 = BeautifulSoup(news_url, "lxml", from_encoding = "utf-8")
            article_body = soup2.find('div', id='articleBodyContents')
            dirt_text = article_body.find_all(text=True)
            cleaned_text = re.sub('[a-zA-Z]', '', str(dirt_text))
            cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                              '', cleaned_text)
            cleaned_text = re.sub('본문 내용    플레이어     플레이어     오류를 우회하기 위한 함수 추가',
                         '', cleaned_text)

            title = soup2.find('h3', id='articleTitle')
            title2 = title.find_all(text=True)
            title2 = re.sub('[a-zA-Z]', '', str(title2))
            title2 = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                              '', title2)
            
        except AttributeError:
            try:
                article = a.find('a')['href']
                news_url = urllib.request.urlopen(article)
                soup2 = BeautifulSoup(news_url, "lxml", from_encoding = "utf-8")
                article_body = soup2.find('div', id='articeBody')
                dirt_text = article_body.find_all(text=True)
                cleaned_text = re.sub('[a-zA-Z]', '', str(dirt_text))
                cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                  '', cleaned_text)
                cleaned_text = re.sub('본문 내용    플레이어     플레이어     오류를 우회하기 위한 함수 추가',
                             '', cleaned_text)

                title = soup2.find('p', attrs={'class':'end_tit'})
                title2 = title.find_all(text=True)
                title2 = re.sub('[a-zA-Z]', '', str(title2))
                title2 = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                  '', title2)
            
            except AttributeError:
                title2 = "Error"
                cleaned_text = "Error"
            
            
        
        
        
        press = a.find('span', {'class': 'press'}).text
        date = a.find('span',{'class':'time'}).text
    
        article_url.append(article)
        press_name.append(press)
        press_time.append(date)
        
        press_title.append(title2)
        press_text.append(cleaned_text)
        
    random_time = random.randint(20, 40) # 이 부분은 네이버뉴스를 크롤링할 때 페이지 넘기는 속도 제어하는 부분
    time.sleep(random_time)

In [41]:
i

2

In [42]:
date = Series(press_time)
name = Series(press_name)
press_url = Series(article_url)
title_sr = Series(press_title)
text_sr = Series(press_text)

url_df = pd.concat([title_sr, name, date, text_sr, press_url], axis=1)
url_df.columns = ['TITLE', 'NAME', 'TIME', 'TEXT', 'URL']

In [43]:
url_df.to_csv(r"D:\test" + file_name + ".csv")

In [44]:
print("==================Finished=================")

==================Finished=================
